# Rain Analysis
### Purpose
This notebook will look at volunteer trends for reporting rain, adressing the Github issue #54

### Author: 
Hamza El-Saawy
### Date: 
2020-06-14
### Update Date: 
2020-06-14

### Inputs 
 - `1.1-circles_to_many_stations_usa_weather_data_20200623005013.txt`

### Output Files
`2.1-cbc_prcp_1900-2018.csv`: A reduced CBC dataset consisiting of only rain (precipitaion) data and an analysis of that data compared to the NOAA GHCN data

## Steps or Proceedures in the notebook 
 - Clean the CBC data
 - Compare to NOAA data
 - Make some plots

## Where the Data will Be Saved 
The project Google Drive, at: https://drive.google.com/drive/folders/1Nlj9Nq-_dPFTDbrSDf94XMritWYG6E2I

## Notes
the flattened NOAA BigQuery drops the `QFLAG` column, so we cannot drop erroneous data and also does not contain the `WT**` `element` values (which can be used alongside the `PRCP` fields to determin precipitation)

Additionally, 1.1 drops rows where `temp_min_value`, `temp_max_value`, `temp_avg`, and `snow` are `nan`, but they could have usable values for `[am|pm]_rain`, since it is much easier to simply annotate if weather happened vs taking measurments.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import scipy.stats as stats

In [3]:
import sklearn.metrics as metrics

In [4]:
sns.set(style="darkgrid")

In [5]:
# get the haversine distance formula fromt the script (w/o executing the '__main__' part)
%run -ni '../scripts/noaa.py'

The cleaned data set, `1.0-rec-initial-data-cleaning.txt`, drops circles with "impossible" temperture, wind, and snow values, which we still find valuable here since we assume that even mistaken/erroneous temp/wind data can still have valuable precipitation data

In [6]:
#
#
#

# drop all stations farther than this threshold (meters)
#  the farthers is ~36km, with the average being 10km
#  16km ~= 10 mi
DISTANCE_THRESHOLD = 15000

# consider stations to consense on a rain value if a fraction of them or more all have the same reading
# at the most abiguous, the fraction will be 0.5, so values are symmetric around 1/2: 0.25 in agreement is the same as 0.75 agreeing on the opposite
AGREEMENT_THRESHOLD = 0.75
AGREEMENT_THRESHOLD = max(AGREEMENT_THRESHOLD, 1 - AGREEMENT_THRESHOLD)
#
#
#

In [7]:
DATA_PATH = '../data/Cloud_Data'
RAW_DATA_PATH = os.path.join(DATA_PATH, 'cbc_effort_weather_1900-2018.txt')
CLN_DATA_PATH = os.path.join(DATA_PATH, '1.0-rec-initial-data-cleaning.txt')
NOAA_DATA_PATH = os.path.join(DATA_PATH, '1.1-circles_to_many_stations_usa_weather_data_20200623005013.txt')
CBC_PRCP_PATH = os.path.join(DATA_PATH, 'cbc_prcp_1900-2018.txt')

In [8]:
raw_data = pd.read_csv(RAW_DATA_PATH, encoding = "ISO-8859-1", sep="\t")
clean_data = pd.read_csv(CLN_DATA_PATH, encoding = "ISO-8859-1", sep="\t")
noaa_data = pd.read_csv(NOAA_DATA_PATH, encoding = "ISO-8859-1", sep="\t").rename(columns={'lat': 'c_lat', 'lon': 'c_lon', 
                                                                                           'id': 's_id', 'latitude': 's_lat', 'longitude': 's_lon'})

/home/hamza/anaconda3/envs/audubon/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/hamza/anaconda3/envs/audubon/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
prcp_data = noaa_data.loc[:, ('count_date',
                              'circle_name', 'country_state', 'circle_id', 'c_lat', 'c_lon',
                              'am_rain', 'pm_rain',
                              's_id', 's_lat', 's_lon',
                              'precipitation_value',
                             )]

## Data Prep

#### stations

In [10]:
# pd.NA preserved int-ness of bools, so they are not converted to floats, and supports three-valued (kleene) logic
prcp_data['s_rain'] = np.where(prcp_data.precipitation_value.isna(), pd.NA, prcp_data.precipitation_value > 0)

#### volunteer records

`[am|pm]_rain` are strings containing `1`:`4`, for heavy, light, none, or unknow rain
If the string contains `4`, then -- regardless of observations in that string (e.g. `2,4`) -- it will be marked as `NaN`  
If the string contains either `1` or `2` in the am or pm, then there was precipitation that day  
If both am and pm are `3`, then there was no precipitation that day  
Else, we mark `nan`

In [11]:
for c in ['am_rain', 'pm_rain']:
    prcp_data.loc[prcp_data[c].isna(), c] = pd.NA
    prcp_data.loc[prcp_data[c].str.contains('4', na=False), c] = pd.NA

In [ ]:
prcp_data['c_rain'] = pd.NA

prcp_data.loc[(prcp_data.am_rain.str.contains('[12]', na=False) | prcp_data.pm_rain.str.contains('[12]', na=False)), 'c_rain'] = True
prcp_data.loc[((prcp_data.am_rain == '3') & (prcp_data.pm_rain == '3')), 'c_rain'] = False

## distance between stations and circles

In [ ]:
prcp_data['distance'] = prcp_data.apply(lambda tt: haversine_formula((tt.c_lat, tt.c_lon), (tt.s_lat, tt.s_lon)), axis=1)

## data formatting

In [ ]:
# drop everything outside of the radius
prcp_data.drop(prcp_data[prcp_data.distance > DISTANCE_THRESHOLD].index, inplace=True)

In [ ]:
prcp_data = prcp_data.loc[:, ['count_date', 
                              'circle_name', 'country_state', 'circle_id', 'c_lat', 'c_lon',
                              's_id', 's_lat', 's_lon', 'distance',
                              'c_rain', 's_rain',]]

undo the left join with the station so each circle/count data occurs once   
this is for efficency reasons: i just want to work with the station data, then add in circle data afterwards  
(I am assumiong circle is unique for each date: no two circles with different lat/lons have the same id)

In [ ]:
# just the circle location information information (no dates)
circle_metadata = prcp_data[['circle_name', 'country_state', 'circle_id', 'c_lat', 'c_lon',]].groupby(['circle_id']).agg('first')
# circle date and location information along with volunteer data
circle_obs = prcp_data[['count_date', 'circle_name', 'country_state', 'circle_id', 
                        'c_lat', 'c_lon','c_rain']].groupby(['circle_id', 'count_date']).agg('first')

In [ ]:
# circle and station multi-index, with only station information
station_obs = prcp_data[['count_date', 'circle_id', 
           's_id', 's_lat', 's_lon', 'distance',
           's_rain',]].set_index(['circle_id', 'count_date', 's_id']).sort_index()

In [ ]:
g = station_obs.groupby(level=['circle_id', 'count_date'])

# Rain Analysis
All the below analyses use the fraction-agreement threshold defined above, and are only for stations within the above-defined distance threshold

In [ ]:
def rain_calc(dfg):
    is_na = dfg.s_rain.isna()
    
    num = dfg.s_rain.size + 0
    num_notna = dfg.s_rain.count() + 0
    num_true = dfg.s_rain.sum() + 0

    return pd.Series({
        # there can be weirdness with boolean not being promoted to ints, so add zero
        'num' : num,
        'num_notna' : num_notna,
        'num_true' : num_true,

        'p' : num_true / num_notna if (num_notna > 0) else np.NaN,
        # theres a sinister bug where stations present twice for the same circle, so dims are not dropped for `dfg.s_rain[...idxmin()]`
        # in that case, the result is a series
        # so,force retention as dataframe and this convoluted bs :/
        # examples: ('9yuvef2', '2010-12-26'), ('9z70n7m', '2008-12-27'), ('djvyywp', '2013-12-24'), ('dp9mpqu', '2012-12-15'), ('dpe0e5b', '2010-12-26'), ...
        'rain_closest' : dfg.loc[[dfg.distance.idxmin()]].s_rain.iloc[0] if (num > 0) else pd.NA,
        'rain_closest_notna' : dfg.loc[[dfg.loc[~is_na, 'distance'].idxmin()]].s_rain.iloc[0] if (num_notna > 0) else pd.NA,
    })


In [ ]:
station_rain = g.apply(rain_calc)
station_rain = station_rain.join(circle_obs)

In [ ]:
station_rain['consensus'] = np.where((station_rain.p >= (1-AGREEMENT_THRESHOLD)) & (station_rain.p <= AGREEMENT_THRESHOLD), 
                                     pd.NA, station_rain.p >= AGREEMENT_THRESHOLD)

## total stats

### the number of stations per circle

In [ ]:
station_rain.num.describe()

In [ ]:
sns.distplot(station_rain.num, kde=False).set_xlabel("number of stations")

### the number of non-NaN stations per circle

In [ ]:
station_rain.num_notna.describe()

In [ ]:
sns.distplot(station_rain.num_notna, kde=False).set_xlabel("number of non-NaN stations")

In [ ]:
sns.jointplot(station_rain.num, station_rain.num_notna/station_rain.num * 100).set_axis_labels("number of stations", 'percent not NaN')

### percent of circles where all stations are missing data

In [ ]:
(station_rain.num_notna == 0).sum() / station_rain.size * 100

### location, location, location

what percent of circles had the closest station as NaN?

In [ ]:
station_rain.rain_closest.isna().sum() / len(station_rain) * 100

what percent had the average value differ from the closest value (ignoring NaNs)?

In [ ]:
# use kleene logical indexing to skip over NAs
(station_rain.rain_closest_notna ^ station_rain.consensus).sum() / len(station_rain) * 100

In [ ]:
station_obs.distance.describe()

In [ ]:
sns.distplot(station_obs.distance, kde=False)

### overall consensus of stations
we use `4 * p(1-p)` to estiamte the "disagreement" amongst stations: this value ranges from `0` (all in agrement) to `1` (evenly split)  
(here, `p` is the fraction of non-NaN stations that are `True` for rain)

In [ ]:
# "disagreement" (4*p*(1-p)) by number of not NA
# horizontal line is AGREEMENT_THRESHOLD, defined above
p = sns.jointplot(station_rain.num_notna, 4 * station_rain.p * (1 - station_rain.p))
p.set_axis_labels("number NaN", '"disgreement [4*p*(1-p)]"')
p.ax_joint.axhline(y = 4 * AGREEMENT_THRESHOLD * (1-AGREEMENT_THRESHOLD))

In [ ]:
# "disagreement" (4*p*(1-p)) by number of not NA
# horizontal line is AGREEMENT_THRESHOLD, defined above
p = sns.jointplot(station_rain.num_notna, 4 * station_rain.p * (1 - station_rain.p))
p.set_axis_labels("number NaN", '"disgreement [4*p*(1-p)]"')
p.ax_joint.axhline(y = 4 * AGREEMENT_THRESHOLD * (1-AGREEMENT_THRESHOLD))

what percent do not meet our dissagreement threshold?

In [ ]:
station_rain.consensus.isna().sum() / len(station_rain) * 100

## the MISSING

In [ ]:
station_rain_na_circle_idx = station_rain.c_rain.isna()

### percent of circles with missing rain observation

In [ ]:
station_rain_na_circle_idx.sum() / len(station_rain) * 100

### percent of circles with both volunteer and all station data are missing

In [ ]:
# joint
(station_rain_na_circle_idx & (station_rain.num_notna == 0)).sum() / len(station_rain) * 100

In [ ]:
# conditional 
(station_rain_na_circle_idx & (station_rain.num_notna == 0)).sum() / station_rain_na_circle_idx.sum() * 100

In [ ]:
# population
(station_rain.num_notna == 0).sum() / station_rain.size * 100

correlation between the two

In [ ]:
g, p, dof, expctd = stats.chi2_contingency(pd.crosstab(station_rain_na_circle_idx, station_rain.num_notna == 0))
g, p

if the circle is missing data, it is much more likely that all other stations will too, when compared to the general population

### percent of circles with both volunteer and the closest station data are missing

In [ ]:
# joint
(station_rain_na_circle_idx & station_rain.rain_closest.isna()).sum()  / len(station_rain) * 100

In [ ]:
# conditional 
(station_rain_na_circle_idx & station_rain.rain_closest.isna()).sum() / station_rain_na_circle_idx.sum() * 100

In [ ]:
# population
station_rain.rain_closest.isna().sum() / len(station_rain) * 100

In [ ]:
g, p, dof, expctd = stats.chi2_contingency(pd.crosstab(station_rain_na_circle_idx, station_rain.rain_closest.isna()))
g, p

if the circle is missing data, it is much **less** likely that the closes station will have missing data, when compared to the general population

### number of stations for circles with missing data

In [ ]:
actl = pd.crosstab(station_rain_na_circle_idx, station_rain.num_notna)
g, p, dof, expctd = stats.chi2_contingency(actl)
g, p

In [ ]:
sns.barplot(np.arange(expctd.shape[1]), actl.loc[True] - expctd[1, :])

it looks like the circle being missing implies that there is only one non-na station?

### disagreement

In [ ]:
# "disagreement" (4*p*(1-p)) by number of not NA
# horizontal line is AGREEMENT_THRESHOLD, defined above
p = sns.jointplot(station_rain.loc[station_rain_na_circle_idx, 'num_notna'], 
                  4 * station_rain.loc[station_rain_na_circle_idx, 'p'] * (1 - station_rain.loc[station_rain_na_circle_idx, 'p']))
p.ax_joint.scatter(station_rain.num_notna, 4 * station_rain.p * (1 - station_rain.p), color='pink', marker='x', alpha=0.5)
p.set_axis_labels("number NaN", '"disgreement [4*p*(1-p)]"')
p.ax_joint.axhline(y = 4 * AGREEMENT_THRESHOLD * (1-AGREEMENT_THRESHOLD))

what percent do not meet our dissagreement threshold?

In [ ]:
# joint
station_rain.loc[station_rain_na_circle_idx, 'consensus'].isna().sum() / len(station_rain) * 100

In [ ]:
# conditional
station_rain.loc[station_rain_na_circle_idx, 'consensus'].isna().sum() / station_rain_na_circle_idx.sum() * 100

In [ ]:
# population
station_rain.consensus.isna().sum() / len(station_rain) * 100

In [ ]:
actl = pd.crosstab(station_rain_na_circle_idx, station_rain.consensus.isna())
g, p, dof, expctd = stats.chi2_contingency(actl)
g, p

In [ ]:
actl - expctd

missing your rain data imples the stations are more likely to consense

# the found

### based on the closest station

In [ ]:
actl = pd.crosstab(station_rain['rain_closest_notna'], station_rain['c_rain'])
actl

In [ ]:
# condition on volunteer data
actl / actl.sum()

In [ ]:
# total joint
actl / actl.to_numpy().sum()

In [ ]:
# accuracy
actl.to_numpy().diagonal().sum() / actl.to_numpy().sum()

In [ ]:
# precision
pr = actl.loc[1,1] / actl.to_numpy()[[1,0], [1,1]].sum()
pr

In [ ]:
# recall
re = actl.loc[1,1] / actl.to_numpy()[[1,1], [1,0]].sum()
re

In [ ]:
## F1
2 * pr * re / (pr + re)

### based on the stations' consensus

In [ ]:
actl = pd.crosstab(station_rain.consensus, station_rain['c_rain'])
actl

In [ ]:
# condition on volunteer data
actl / actl.sum()

In [ ]:
# total joint
actl / actl.to_numpy().sum()

In [ ]:
# accuracy
actl.to_numpy().diagonal().sum() / actl.to_numpy().sum()

In [ ]:
# precision
pr = actl.loc[1,1] / actl.to_numpy()[[1,0], [1,1]].sum()
pr

In [ ]:
# recall
re = actl.loc[1,1] / actl.to_numpy()[[1,1], [1,0]].sum()
re

In [ ]:
## F1
2 * pr * re / (pr + re)

### did atleast one station aggree?

In [ ]:
actl = pd.crosstab(station_rain.num_true > 1, station_rain['c_rain'])
actl

In [ ]:
# condition on volunteer data
actl / actl.sum()

In [ ]:
# total joint
actl / actl.to_numpy().sum()

In [ ]:
# accuracy
actl.to_numpy().diagonal().sum() / actl.to_numpy().sum()

In [ ]:
# precision
pr = actl.loc[1,1] / actl.to_numpy()[[1,0], [1,1]].sum()
pr

In [ ]:
# recall
re = actl.loc[1,1] / actl.to_numpy()[[1,1], [1,0]].sum()
re

In [ ]:
## F1
2 * pr * re / (pr + re)